# Add Image Metrics to an Existing Table

<div style="display: inline-flex; align-items: center; gap: 10px;">
        <a href="https://colab.research.google.com/github/3lc-ai/3lc-examples/blob/main/tutorials/add-image-metrics.ipynb"
        target="_blank"
            style="background-color: transparent; text-decoration: none; display: inline-flex; align-items: center;
            padding: 5px 10px; font-family: Arial, sans-serif;"> <img
            src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="height: 30px;
            vertical-align: middle;box-shadow: none;"/>
        </a> <a href="https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/add-image-metrics.ipynb"
            style="text-decoration: none; display: inline-flex; align-items: center; background-color: #ffffff; border:
            1px solid #d1d5da; border-radius: 8px; padding: 2px 10px; color: #333; font-family: Arial, sans-serif;">
            <svg aria-hidden="true" focusable="false" role="img" class="octicon octicon-mark-github" viewBox="0 0 16 16"
            width="20" height="20" fill="#333"
            style="display:inline-block;user-select:none;vertical-align:text-bottom;overflow:visible; margin-right:
            8px;">
                <path d="M8 0c4.42 0 8 3.58 8 8a8.013 8.013 0 0 1-5.45 7.59c-.4.08-.55-.17-.55-.38 0-.27.01-1.13.01-2.2
                0-.75-.25-1.23-.54-1.48 1.78-.2 3.65-.88 3.65-3.95 0-.88-.31-1.59-.82-2.15.08-.2.36-1.02-.08-2.12 0
                0-.67-.22-2.2.82-.64-.18-1.32-.27-2-.27-.68 0-1.36.09-2 .27-1.53-1.03-2.2-.82-2.2-.82-.44 1.1-.16
                1.92-.08 2.12-.51.56-.82 1.28-.82 2.15 0 3.06 1.86 3.75 3.64 3.95-.23.2-.44.55-.51
                1.07-.46.21-1.61.55-2.33-.66-.15-.24-.6-.83-1.23-.82-.67.01-.27.38.01.53.34.19.73.9.82 1.13.16.45.68
                1.31 2.69.94 0 .67.01 1.3.01 1.49 0 .21-.15.45-.55.38A7.995 7.995 0 0 1 0 8c0-4.42 3.58-8 8-8Z"></path>
            </svg> <span style="vertical-align: middle; color: #333;">Open in GitHub</span>
        </a>
</div>

In this example, we will first write a simple table containing a single column of image paths.
We will then write a second table, containing the image paths from the first table, and extended with new columns containing image metrics.

In [1]:
from collections import defaultdict
from pathlib import Path

import numpy as np
import tlc
from PIL import Image, ImageStat

from tools import add_columns_to_table


## Write the initial table

In [2]:
data_path = Path("../data/coco128/images").absolute().as_posix()
dataset_name = "coco128"
project_name = "add-image-metrics"

table = tlc.Table.from_image_folder(
    data_path,
    table_name="initial",
    dataset_name=dataset_name,
    project_name=project_name,
    include_label_column=False,
    add_weight_column=False,
    description="Initial table with images from COCO128 dataset",
)

## Extend the table with image metrics

In [3]:
def compute_image_metrics(image_path: str):
    """Return a dict of image metrics for the given image path."""
    image = Image.open(image_path)
    width, height = image.size
    pixels = np.array(image)

    # Convert to grayscale for some metrics
    grayscale_image = image.convert("L")
    stat = ImageStat.Stat(grayscale_image)

    # Compute brightness (average grayscale value)
    brightness = stat.mean[0]

    # Compute contrast (standard deviation of grayscale values)
    contrast = stat.stddev[0]

    # Compute average RGB values
    try:
        avg_r = np.mean(pixels[:, :, 0])
        avg_g = np.mean(pixels[:, :, 1])
        avg_b = np.mean(pixels[:, :, 2])
    except IndexError: # Image is grayscale
        avg_r = avg_g = avg_b = 0

    return {
        "width": width,
        "height": height,
        "brightness": brightness,
        "contrast": contrast,
        "average_red": avg_r,
        "average_green": avg_g,
        "average_blue": avg_b,
    }

In [ ]:

schemas = {
    "width": tlc.Schema(value=tlc.Int32Value(), writable=False, sample_type="hidden"),
    "height": tlc.Schema(value=tlc.Int32Value(), writable=False, sample_type="hidden"),
    "brightness": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
    "contrast": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
    "average_red": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
    "average_green": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
    "average_blue": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
}

# Iterate through the input table, compute metrics for each image, and add the metrics to the extended table
new_columns = defaultdict(list)
for row in table.table_rows:
    image_path = row["image"]
    metrics = compute_image_metrics(image_path)
    for column_name, value in metrics.items():
        new_columns[column_name].append(value)

extended_table = add_columns_to_table(
    table,
    new_columns,
    schemas,
    output_table_name="added-image-metrics",
    description="COCO128 dataset with added image metrics",
)

print(extended_table[0].keys())  # Notice only the "image" column is present in the "sample-view" of the table
print(extended_table.table_rows[0].keys())  # Notice all the columns are present in the "row-view" of the table